In [75]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from tqdm import tqdm
import time 
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 255)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
df = pd.read_csv('./datas/data.csv')
df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of v...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen-...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well b...",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br...",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situ...",positive


In [21]:
df = df.iloc[:10000]
df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of v...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen-...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well b...",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br...",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situ...",positive


In [4]:
df.shape

(10000, 2)

In [5]:
df['sentiment'].value_counts()

sentiment
positive    5028
negative    4972
Name: count, dtype: int64

In [6]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [7]:
df.duplicated().sum()

17

In [22]:
# The inplace=True parameter ensures that the changes are made directly to the existing DataFrame without creating a new copy.
df.drop_duplicates(inplace=True)

In [23]:
df.duplicated().sum()

0

# Basic Preprocessing

In [24]:
import re
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'',text)

In [25]:
df['review'] = df['review'].apply(remove_html_tags)

In [26]:
import re
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'',text)

In [27]:
df['review'] = df['review'].apply(remove_url)

In [28]:
# performing lowercasing operation
df['review'] = df['review'].str.lower()

In [29]:
df.head()

,review,sentiment
0,"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, whi...",positive
1,"a wonderful little production. the filming technique is very unassuming- very old-time-bbc fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. the actors are extremely well chosen- michael sheen not only ...",positive
2,"i thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. the plot is simplistic, but the dialogue is witty and the characters are likable (even the well b...",positive
3,"basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.ok, first of al...",negative
4,"petter mattei's ""love in the time of money"" is a visually stunning film to watch. mr. mattei offers us a vivid portrait about human relations. this is a movie that seems to be telling us what money, power and success do to people in the different situ...",positive


In [30]:
# remove puncuation
import string, time
string.punctuation
exclude = string.punctuation

In [31]:
def remove_punc(text):
    for char in exclude:
        text = text.replace(char, '')
    return text

In [32]:
df['review'] = df['review'].apply(remove_punc)

In [33]:
df['review'][4]

'petter matteis love in the time of money is a visually stunning film to watch mr mattei offers us a vivid portrait about human relations this is a movie that seems to be telling us what money power and success do to people in the different situations we encounter this being a variation on the arthur schnitzlers play about the same theme the director transfers the action to the present time new york where all these different characters meet and connect each one is connected in one way or another to the next person but no one seems to know the previous point of contact stylishly the film has a sophisticated luxurious look we are taken to see how these people live and the world they live in their own habitatthe only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits a big city is not exactly the best place in which human relations find sincere fulfillment as one discerns is the case with most of the people we encounterthe acting i

In [34]:
# removing stopwords
sw_list = stopwords.words('english')
def removing_stopwords(text):
    new_text = []
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return ' '.join(x)

In [35]:
df['review'] = df['review'].apply(removing_stopwords)

In [36]:
df.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz episode youll hooked right exactly happened methe first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches...,positive
1,wonderful little production filming technique unassuming oldtimebbc fashion gives comforting sometimes discomforting sense realism entire piece actors extremely well chosen michael sheen got polari voices pat truly see seam...,positive
2,thought wonderful way spend time hot summer weekend sitting air conditioned theater watching lighthearted comedy plot simplistic dialogue witty characters likable even well bread suspected serial killer may disappointed reali...,positive
3,basically theres family little boy jake thinks theres zombie closet parents fighting timethis movie slower soap opera suddenly jake decides become rambo kill zombieok first youre going make film must decide thriller drama ...,negative
4,petter matteis love time money visually stunning film watch mr mattei offers us vivid portrait human relations movie seems telling us money power success people different situations encounter variation arthur schnitzlers play...,positive


In [37]:
# Seperating the X and y data

In [49]:
X = df.iloc[:,0:1]
y = df['sentiment']

In [51]:
X.head()

,review
0,one reviewers mentioned watching 1 oz episode youll hooked right exactly happened methe first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches...
1,wonderful little production filming technique unassuming oldtimebbc fashion gives comforting sometimes discomforting sense realism entire piece actors extremely well chosen michael sheen got polari voices pat truly see seam...
2,thought wonderful way spend time hot summer weekend sitting air conditioned theater watching lighthearted comedy plot simplistic dialogue witty characters likable even well bread suspected serial killer may disappointed reali...
3,basically theres family little boy jake thinks theres zombie closet parents fighting timethis movie slower soap opera suddenly jake decides become rambo kill zombieok first youre going make film must decide thriller drama ...
4,petter matteis love time money visually stunning film watch mr mattei offers us vivid portrait human relations movie seems telling us money power success people different situations encounter variation arthur schnitzlers play...


Here we need to conver the the positive and negative sentiment in terms of 1 and 0

In [55]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y = encoder.fit_transform(y)

In [56]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [61]:
# test and train split
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [62]:
len(X_train),len(y_train),len(X_test),len(y_test)

(7986, 7986, 1997, 1997)

## Applying Bag of Words

In [63]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [66]:
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

In [68]:
X_train_bow.shape,X_test_bow.shape

((7986, 72694), (1997, 72694))

# Using ML model Naive Bayes

In [69]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(X_train_bow,y_train)

GaussianNB()

In [70]:
from sklearn.metrics import accuracy_score,confusion_matrix
y_pred =  gnb.predict(X_test_bow)

accuracy_score(y_test,y_pred)

0.656484727090636

In [71]:
confusion_matrix(y_test,y_pred)

array([[696, 256],
       [430, 615]], dtype=int64)

# Random Forest Classifier


In [76]:
rf = RandomForestClassifier()
rf.fit(X_train_bow, y_train)

# Predict using the model
y_pred = rf.predict(X_test_bow)

# Simulate a loading bar for the accuracy calculation
for _ in tqdm(range(100), desc="Calculating Accuracy", ncols=80):
    time.sleep(0.02)  # Simulating computation time

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Calculating Accuracy: 100%|███████████████████| 100/100 [00:02<00:00, 48.60it/s]

Accuracy: 0.8392588883324987


In [79]:
# only taking the 3000 unique words
cv = CountVectorizer(max_features=3000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()


df = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8332498748122183

# N_grams

In [80]:
cv = CountVectorizer(ngram_range=(2,2),max_features=5000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()


df = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.7315973960941412

# TFIDF

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review'])


rf = RandomForestClassifier()

rf.fit(X_train_tfidf,y_train)
y_pred = rf.predict(X_test_tfidf)

accuracy_score(y_test,y_pred)
     

0.8537806710065098

 # Word to Vec

In [84]:
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [92]:
dataset = pd.read_csv('./datas/data.csv')
dataset.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of v...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen-...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well b...",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br...",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situ...",positive


In [93]:
# preprocessing 
dataset['review'] =  dataset['review'].apply(remove_html_tags)
dataset['review'] =  dataset['review'].apply(remove_punc)
dataset['review'] =  dataset['review'].apply(remove_url)
# dataset['review'] =  dataset['review'].apply(removing_stopwords)


In [106]:
corpus = [sent_tokenize(value) for value in dataset['review']]
# raw_sent = sent_tokenize(corpus)

story = []

for sent in corpus:
    for values in sent:
        story.append(simple_preprocess(values))

In [123]:
# story

In [108]:
# windo means the model will take that number of word before and after it
# min_count = 2 means that only words that appear at least twice in the corpus will be considered in the vocabulary
model = gensim.models.Word2Vec(
    window = 10,
    min_count = 2
)

In [109]:
model.build_vocab(story)

In [110]:

model.train(story,total_examples = model.corpus_count,epochs = model.epochs)

(41027782, 53829365)

In [112]:
model.wv.most_similar('good')

[('decent', 0.727938175201416),
 ('great', 0.71223384141922),
 ('bad', 0.6999435424804688),
 ('nice', 0.6445486545562744),
 ('cool', 0.626274824142456),
 ('fine', 0.6242803931236267),
 ('ok', 0.5922934412956238),
 ('okay', 0.5790241956710815),
 ('solid', 0.5650421380996704),
 ('impressive', 0.5484288930892944)]

In [113]:
vec = model.wv.get_normed_vectors()
vec

array([[ 6.0585815e-02, -9.0457022e-02,  2.1853958e-01, ...,
        -1.6016345e-01,  5.1838879e-02, -6.5946065e-02],
       [ 1.4178835e-01,  8.7207220e-02,  1.7810951e-01, ...,
        -1.7889884e-04,  9.1884695e-02, -1.3100395e-01],
       [-1.6470229e-02,  8.4398501e-02,  3.6520783e-02, ...,
         1.0332519e-01,  1.4263925e-01,  3.3073511e-03],
       ...,
       [-8.3900794e-02,  4.3512654e-02, -2.3028876e-03, ...,
        -1.0425045e-02, -1.0298126e-01, -7.1509138e-02],
       [ 1.5695916e-02,  6.3220061e-02,  7.6186620e-02, ...,
        -6.7998998e-02, -7.6729387e-02, -1.2050498e-01],
       [-9.9053137e-02,  6.1136566e-02,  5.9854209e-02, ...,
         2.0267922e-02,  8.9396305e-02,  1.4543556e-01]], dtype=float32)

In [125]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y = encoder.fit_transform(dataset['sentiment'])
y

array([1, 1, 1, ..., 0, 0, 0])

In [146]:
def create_feature_vector(review, model):
    # Use the provided list of tokens directly (no need to split again)
    vectors = []
    for word in review:
        if word in model.wv:  # Check if the word is in the model's vocabulary
            vectors.append(model.wv[word])  # Get the word's vector
    if vectors:
        return np.mean(vectors, axis=0)  # Return the mean of word vectors (review vector)
    else:
        return np.zeros(model.vector_size)

In [139]:
story[0:5]

[['one',
  'of',
  'the',
  'other',
  'reviewers',
  'has',
  'mentioned',
  'that',
  'after',
  'watching',
  'just',
  'oz',
  'episode',
  'youll',
  'be',
  'hooked',
  'they',
  'are',
  'right',
  'as',
  'this',
  'is',
  'exactly',
  'what',
  'happened',
  'with',
  'methe',
  'first',
  'thing',
  'that',
  'struck',
  'me',
  'about',
  'oz',
  'was',
  'its',
  'brutality',
  'and',
  'unflinching',
  'scenes',
  'of',
  'violence',
  'which',
  'set',
  'in',
  'right',
  'from',
  'the',
  'word',
  'go',
  'trust',
  'me',
  'this',
  'is',
  'not',
  'show',
  'for',
  'the',
  'faint',
  'hearted',
  'or',
  'timid',
  'this',
  'show',
  'pulls',
  'no',
  'punches',
  'with',
  'regards',
  'to',
  'drugs',
  'sex',
  'or',
  'violence',
  'its',
  'is',
  'hardcore',
  'in',
  'the',
  'classic',
  'use',
  'of',
  'the',
  'wordit',
  'is',
  'called',
  'oz',
  'as',
  'that',
  'is',
  'the',
  'nickname',
  'given',
  'to',
  'the',
  'oswald',
  'maximum',
  

In [147]:
# Assuming df['review'] contains your raw reviews
# Apply simple_preprocess to each review to tokenize and clean the text
processed_reviews = dataset['review'].apply(lambda x: simple_preprocess(x))

# Create feature vectors for each review using Word2Vec
X = np.array([create_feature_vector(review, model) for review in processed_reviews])


In [150]:
X.shape,y.shape

((50000, 100), (50000,))

In [151]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [152]:
len(X_train),len(X_test),len(y_train),len(y_test)

(40000, 10000, 40000, 10000)

In [154]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

y_pred = rf.predict(X_test)


accuracy = accuracy_score(y_test,y_pred)
print(accuracy)

0.8239
